In [12]:
a = generate_tree(5)

In [13]:
a

False

In [14]:
import random

# Define node types
NODE_TYPES = ['AND', 'OR', 'NOT', 'A', 'B', 'C']

# Define maximum depth of the tree
MAX_DEPTH = 3

class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

def and_func(x, y):
    return x and y

def or_func(x, y):
    return x or y

def not_func(x):
    return not x

def nor_func(x, y):
    return not (x or y)

def create_tree(depth):
    if depth <= 0 or random.random() < 0.1:
        # Terminal node, return either 0 or 1 randomly
        return Node(random.choice(['A', 'B', 'C']))
    else:
        # Non-terminal node, choose a function randomly
        node_type = random.choice(['AND', 'OR', 'NOT'])
        node = Node(node_type)
        # Recursively create children nodes
        for _ in range(2 if node_type in ['AND', 'OR'] else 1):
            node.children.append(create_tree(depth - 1))
        return node

def print_tree(node, indent=0):
    if node is not None:
        print('-' * indent + node.value)
        for child in node.children:
            print_tree(child, indent + 2)

def calcular_profundidades(nodo):
    if nodo.children == []:
        return 1
    else:
        return 1 + max([calcular_profundidades(child) for child in nodo.children])

def evaluar_arbol(arbol, a, b, c):
    # funcion recursiva que evalua el arbol, solo evalua para 1 renglon de la tabla de verdad
    if arbol.value == 'A':
        return a
    elif arbol.value == 'B':
        return b
    elif arbol.value == 'C':
        return c
    elif arbol.value == 'AND':
        return and_func(evaluar_arbol(arbol.children[0], a, b, c), evaluar_arbol(arbol.children[1], a, b, c))
    elif arbol.value == 'OR':
        return or_func(evaluar_arbol(arbol.children[0], a, b, c), evaluar_arbol(arbol.children[1], a, b, c))
    elif arbol.value == 'NOT':
        return not_func(evaluar_arbol(arbol.children[0], a, b, c))
    else:
        raise ValueError('Invalid node value')
    
def evaluar_arbol_completo(arbol):
    # funcion que evalua el arbol para todas las combinaciones de la tabla de verdad
    for a in [True, False]:
        for b in [True, False]:
            for c in [True, False]:
                print(f'{a} {b} {c} -> {evaluar_arbol(arbol, a, b, c)}')   

def mutacion(arbol, prob=0.1):
    # funcion que muta un arbol
    # crea un arbol nuevo
    # recorre el arbol original y muta con probabilidad 0.1
    # Mutacion sustituye el nodo por el arbol creado
    
    if random.random() < prob:
        prob = 0
        return create_tree(MAX_DEPTH)
    else:
        nuevo_arbol = Node(arbol.value)
        for child in arbol.children:
            nuevo_arbol.children.append(mutacion(child))
        return nuevo_arbol

def cruzamiento(arbol1, arbol2, prob = 0.8):
    # Identifica los puntos de cruza en cada padre
    # intercambiar los nodos entre los padres
    # retorna 2 hijos
    if random.random() > prob:
        return arbol2, arbol1
    else:
        nodos_a1 = num_nodos(arbol1)
        nodos_a2 = num_nodos(arbol2)
        punto_cruza1 = random.randint(1, nodos_a1)
        punto_cruza2 = random.randint(1, nodos_a2)
        hijo1 = cruzar(arbol1, arbol2, punto_cruza1, punto_cruza2)
        hijo2 = cruzar(arbol2, arbol1, punto_cruza2, punto_cruza1)
        return hijo1, hijo2

def cruzar(arbol1, arbol2, punto_cruza1, punto_cruza2):
    if arbol1.children == []:
        
        
        

def num_nodos(arbol):
    if arbol.children == []:
        return 1
    else:
        return 1 + sum([num_nodos(child) for child in arbol.children])
    



# Create a tree with maximum depth of 3
individual = create_tree(3)
print_tree(individual)
print(calcular_profundidades(individual))


OR
--NOT
----OR
------B
------B
--NOT
----A
4


In [15]:
individual = create_tree(3)
individual2 = create_tree(3)

In [16]:
hijo1, hijo2 = cruzamiento(individual, individual2)
# hijo2 = cruzamiento(individual2, individual)

IndexError: list index out of range

In [5]:
print_tree(individual)
print("Si padre: \n")
print_tree(individual2)

AND
--NOT
----AND
------C
------C
--AND
----NOT
------C
----NOT
------C
Si padre: 

OR
--AND
----NOT
------B
----NOT
------C
--NOT
----OR
------C
------C


In [7]:
print_tree(hijo1)
print("Si padre: \n")
print_tree(hijo2)

AND
--AND
----NOT
------B
----NOT
------C
--NOT
----OR
------C
------C
Si padre: 

OR
--AND
----AND
------C
------C
--NOT
----OR
------C


In [9]:
print_tree(individual)

OR
--NOT
----NOT
------A
--AND
----C
----NOT
------C


In [10]:
num_nodos(individual)

8